# プロジェクト - 航空会社AIアシスタント

航空会社のAIカスタマーサポートアシスタントを作成するために学んだことをまとめます

In [ ]:
# 輸入

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# 初期化

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# 別の方法として、Openaiの代わりにOllamaを使用したい場合
# Ollamaが地元で走っていることを確認してください（Week1/day2エクササイズを参照）、次の2行の発見を解消します
# Model = "llama3.2"
# openai = openai（base_url = 'http：// localhost：11434/v1'、api_key = 'ollama'）


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# この関数は、最新のグラデーションアップデートを利用しているため、私のビデオの機能よりもかなりシンプルに見えます。

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## ツール

ツールは、フロンティアLLMSが提供する非常に強力な機能です。

ツールを使用すると、関数を記述し、LLMにその機能をその応答の一部として呼び出すことができます。

ほとんど不気味に聞こえます。マシンでコードを実行する力を与えていますか？

まあ、ちょっと。

In [ ]:
# 役立つ機能を作成することから始めましょう

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Berlin")

In [ ]:
# 私たちの機能を説明するために必要な特定の辞書構造があります。

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# これはツールのリストに含まれています。

tools = [{"type": "function", "function": price_function}]

## Openaiを取得してツールを使用します

Openaiが「私たちのツールを呼び出す」ことを許可するための厄介なものがいくつかあります

私たちが実際に行っていることは、LLMにツールを実行することを望んでいることを知らせる機会を与えることです。

新しいチャット関数がどのように見えるかは次のとおりです。

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# その関数をhandle_tool_call：書く必要があります：

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()